In [1]:
import torch
import torch.optim as optim
import torch
from torch.utils.data import DataLoader
import model
import datasets

In [ ]:
# Make sure to use the GPU. The following line is just a check to see if GPU is availables
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
# load your dataset and dataloader
# feel free to change header of bird_dataset class
root = 'path to images/'
train_dataset = bird_dataset(root,'train_list.txt')
test_dataset = bird_dataset(root,'test_list.txt')

# Fill in optional arguments to the dataloader as you need it
train_dataloader = DataLoader(train_dataset)
test_dataloader = DataLoader(test_dataset)

In [ ]:
# Create NN model object
nn_model = model.my_Net()
nn_model.to(device)


In [ ]:
# Create loss functions, optimizers


In [ ]:
# train your model
# For each epoch iterate over your dataloaders/datasets, pass it to your NN model, get output, calculate loss and
# backpropagate using optimizer


In [ ]:
# Save your model/best model